In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

In [2]:
#  Import and read the charity_data.csv.
cleaned_df = pd.read_csv("Resources/cleaned_up_dataset.csv")
cleaned_df.head()

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,hit,genre
0,4pR4oQJULf7FDB54TleMyH,Limoblaze;Lecrae;Happi,Jireh (My Provider),Jireh (My Provider),64,168000,False,0.443,0.778,0,...,0,0.2660,0.241,0.000000,0.2150,0.628,128.250,5,1.0,Hip_Hop_R&B
1,4Ds4bq6aqOSAp1T7DikOi4,Criolo,Ainda Há Tempo,Ainda Há Tempo,44,318360,False,0.580,0.800,6,...,1,0.3450,0.281,0.000000,0.0708,0.483,81.303,4,0.0,Hip_Hop_R&B
2,7nYbKsvhM88mHCmFsAIQVp,Criolo,Fellini,Fellini,42,171989,False,0.805,0.746,1,...,0,0.1850,0.238,0.000000,0.2150,0.717,127.922,4,0.0,Hip_Hop_R&B
3,7gVNP7rI9UBZndge0ulKfL,Criolo,Nó na Orelha,Lion Man,46,205440,False,0.582,0.660,7,...,0,0.0544,0.399,0.000000,0.1270,0.484,83.789,4,0.0,Hip_Hop_R&B
4,1hLvWelTny8vttEEZIXVjw,Criolo,Espiral de Ilusão,Espiral de Ilusão,44,220800,False,0.776,0.314,2,...,1,0.0340,0.881,0.000723,0.1390,0.476,124.055,4,0.0,Hip_Hop_R&B


In [3]:
# Drops track_id, artists, album_name, track_name, popularity, explicit
cleaned_df.drop(['track_id','artists','album_name', 'track_name', 'popularity', 'explicit'], axis = 1, inplace=True)
cleaned_df.head()

,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,hit,genre
0,168000,0.443,0.778,0,-7.564,0,0.2660,0.241,0.000000,0.2150,0.628,128.250,5,1.0,Hip_Hop_R&B
1,318360,0.580,0.800,6,-7.528,1,0.3450,0.281,0.000000,0.0708,0.483,81.303,4,0.0,Hip_Hop_R&B
2,171989,0.805,0.746,1,-5.211,0,0.1850,0.238,0.000000,0.2150,0.717,127.922,4,0.0,Hip_Hop_R&B
3,205440,0.582,0.660,7,-4.988,0,0.0544,0.399,0.000000,0.1270,0.484,83.789,4,0.0,Hip_Hop_R&B
4,220800,0.776,0.314,2,-9.513,1,0.0340,0.881,0.000723,0.1390,0.476,124.055,4,0.0,Hip_Hop_R&B


In [4]:
# Generate our categorical variable lists
cleaned_cat = cleaned_df.dtypes[cleaned_df.dtypes == 'object'].index.tolist()
cleaned_cat

['genre']

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(cleaned_df[cleaned_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(cleaned_cat)
encode_df.head()

C:\Users\admin\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,genre_Cinema,genre_Classical,genre_Country,genre_Dance,genre_Foreign,genre_Hip_Hop_R&B,genre_Indie,genre_Jazz_Blues,genre_Kids,genre_Latin,genre_Pop,genre_Reggae,genre_Rock
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Merge the dataframes and drop genre
cleaned_df = cleaned_df.merge(encode_df, left_index=True, right_index=True).drop("genre",1)

C:\Users\admin\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [7]:
cleaned_df.head()

,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,genre_Dance,genre_Foreign,genre_Hip_Hop_R&B,genre_Indie,genre_Jazz_Blues,genre_Kids,genre_Latin,genre_Pop,genre_Reggae,genre_Rock
0,168000,0.443,0.778,0,-7.564,0,0.2660,0.241,0.000000,0.2150,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,318360,0.580,0.800,6,-7.528,1,0.3450,0.281,0.000000,0.0708,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,171989,0.805,0.746,1,-5.211,0,0.1850,0.238,0.000000,0.2150,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,205440,0.582,0.660,7,-4.988,0,0.0544,0.399,0.000000,0.1270,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,220800,0.776,0.314,2,-9.513,1,0.0340,0.881,0.000723,0.1390,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Split our preprocessed data into our features and target arrays
y = cleaned_df['hit'].values
X = cleaned_df.drop(['hit'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

C:\Users\admin\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 54
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 54)                1458      
                                                                 
 dense_1 (Dense)             (None, 30)                1650      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 3,139
Trainable params: 3,139
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [13]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.5291 - accuracy: 0.7437
Epoch 2/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.5158 - accuracy: 0.7489
Epoch 3/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.5102 - accuracy: 0.7530
Epoch 4/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.5062 - accuracy: 0.7553
Epoch 5/100
2191/2204 [============================>.] - ETA: 0s - loss: 0.5032 - accuracy: 0.7566
Epoch 5: saving model to checkpoints\weights.05.hdf5
2204/2204 [==============================] - 3s 1ms/step - loss: 0.5034 - accuracy: 0.7564
Epoch 6/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.5010 - accuracy: 0.7578
Epoch 7/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.4990 - accuracy: 0.7605
Epoch 8/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.4970 - accuracy: 0.7601
Epoch 9/100
2204/2204 [=============

2204/2204 [==============================] - 3s 1ms/step - loss: 0.4621 - accuracy: 0.7793
Epoch 64/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.4619 - accuracy: 0.7793
Epoch 65/100
2184/2204 [============================>.] - ETA: 0s - loss: 0.4616 - accuracy: 0.7787
Epoch 65: saving model to checkpoints\weights.65.hdf5
2204/2204 [==============================] - 3s 1ms/step - loss: 0.4615 - accuracy: 0.7786
Epoch 66/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.4608 - accuracy: 0.7801
Epoch 67/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.4610 - accuracy: 0.7797
Epoch 68/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.4605 - accuracy: 0.7816
Epoch 69/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.4605 - accuracy: 0.7811
Epoch 70/100
2177/2204 [============================>.] - ETA: 0s - loss: 0.4600 - accuracy: 0.7811
Epoch 70: saving model to checkpoints\weight

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

735/735 - 1s - loss: 0.5148 - accuracy: 0.7535 - 755ms/epoch - 1ms/step
Loss: 0.5148369073867798, Accuracy: 0.7535319328308105


In [16]:
# Export model to HDF5 file
nn.save("Spotify_ml.h5")